# Setup

In [1]:
import pandas as pd
import numpy as np
import pickle
import time

from custom_methods import model_eval

datapath = '../Data/'

startTime = time.time()

In [2]:
filename = 'ann_ready.pickle'
infile = open(datapath+filename,'rb')
data = pickle.load(infile)
X_train = data['X_train']
Y_train = data['Y_train']
X_test = data['X_test']
Y_test = data['Y_test']
infile.close()

print(type(X_train))
X_train

<class 'numpy.ndarray'>


array([[[-0.28867513, -0.28867513,  0.        , ..., -1.08012345,
         -1.08012345, -1.08012345],
        [ 0.        ,  0.        , -0.28867513, ..., -1.08012345,
         -1.08012345, -1.08012345],
        [ 0.        ,  0.        , -0.28867513, ..., -1.08012345,
         -1.08012345, -1.08012345],
        ...,
        [ 0.        ,  0.        , -0.28867513, ...,  0.        ,
         -0.49746834,  0.28867513],
        [ 0.        ,  0.        , -0.28867513, ...,  0.        ,
         -0.49746834,  0.28867513],
        [ 0.        ,  0.        , -0.28867513, ...,  0.        ,
         -0.64951905,  0.28867513]],

       [[-0.28867513, -0.28867513,  0.        , ..., -1.08012345,
         -1.08012345, -1.08012345],
        [ 0.        ,  0.        , -0.28867513, ..., -1.08012345,
         -1.08012345, -1.08012345],
        [ 0.        ,  0.        , -0.28867513, ..., -1.08012345,
         -1.08012345, -1.08012345],
        ...,
        [ 0.        ,  0.        , -0.28867513, ...,  

# Model

## Set Initial Bias

## Build Model

In [13]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping

early_stopping= EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=2,
    restore_best_weights=True
)

model = Sequential()

# Create input layer + LSTM
model.add(
    LSTM(
        units = 64,
        input_shape = (X_train.shape[1], X_train.shape[2]),
    )
)

# Create output layer
model.add(
    Dense(
        units = 1, 
        activation='sigmoid', 
    )
)

# Compile ANN
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

# Print model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                20992     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 21,057
Trainable params: 21,057
Non-trainable params: 0
_________________________________________________________________
None


## Class Weight

In [14]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
# pos = Y_train.sum().values[0]
pos = 1
total = len(Y_train)
neg = total - pos

weight_for_F = (1 / neg) * (total / 2.0)
weight_for_T = (1 / pos) * (total / 2.0)

class_weight = {False: weight_for_F, True: weight_for_T}

print('Weight for class F: {:.2f}'.format(weight_for_F))
print('Weight for class T: {:.2f}'.format(weight_for_T))

Weight for class F: 0.54
Weight for class T: 6.50


## Fit

In [15]:
BATCH_SIZE = 1
EPOCHS = 5


model.fit(
    x = X_train, 
    y = Y_train, 
    batch_size = BATCH_SIZE, 
    epochs = EPOCHS,
    class_weight = class_weight,
    callbacks=[callback]
)

Epoch 1/5
13/13 [==============================] - 0s 9ms/step - loss: 0.3425 - accuracy: 0.6154
Epoch 2/5
13/13 [==============================] - 0s 9ms/step - loss: 0.1971 - accuracy: 0.9231
Epoch 3/5
13/13 [==============================] - 0s 8ms/step - loss: 0.0956 - accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 8ms/step - loss: 0.0429 - accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 8ms/step - loss: 0.0239 - accuracy: 1.0000


In [18]:
predictions = model.predict(x = X_test)
predictions = pd.Series(np.transpose(predictions)[0], name='prediction')
predictions = pd.concat([Y_test.reset_index(), predictions], axis=1, join='inner', ignore_index=False)
predictions.head()

,SPA_PER_ID,CMIS_MATCH,prediction
0,64412,False,0.007906
1,124687,False,0.008017
2,142930,False,0.547152
3,228452,False,0.006629
4,258799,False,0.125511


# Time and Save

In [ ]:
filename = 'lstm_predictions.pickle'
outfile = open(datapath + filename, 'wb')
pickle.dump(predictions, outfile)
outfile.close()

In [ ]:
from custom_methods.calc_time import calc_time_from_sec

calc_time_from_sec(time.time()-startTime)